In [ ]:
pip install fuzzywuzzy

In [ ]:
pip install pandas

In [ ]:
pip install sqlalchemy

In [ ]:
pip install psycopg2

In [ ]:
pip install re

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from fuzzywuzzy import process
import os
import re

In [ ]:
#login for the database and server
user = 'your_user'
password = 'your_password'
host = 'localhost'
port_number = '5432'
database_name = 'Haven'
connection_string = f'postgresql://{user}:{password}@{host}:{port_number}/{database_name}'
engine = create_engine(connection_string) 

In [ ]:
#making new function for loading the data from postgres
def load_data(table_name): #plug in relevant names
    return pd.read_sql_table(table_name, engine)

In [ ]:
#full code that recognizes duplicates both within a table itself and with other tables
def deduplicate_data(df, column_name, threshold=90):
    # Convert all values in the column to strings
    df[column_name] = df[column_name].astype(str)
    
    # Fill missing values with a placeholder string
    df[column_name].fillna('', inplace=True)
    
    unique_names = pd.Series(df[column_name].unique())
    matches = unique_names.apply(lambda x: process.extract(x, unique_names, limit=2))
    matches = matches.apply(lambda x: [(m[0], m[1]) for i, m in enumerate(x) if m[1] >= threshold and m[0] != unique_names.iloc[i]])
    
    # Consolidate duplicates
    consolidated = {}
    for i, match_list in enumerate(matches):
        for match in match_list:
            original = unique_names[i]
            duplicate = match[0]
            if duplicate in consolidated:
                consolidated[duplicate].add(original)
            else:
                consolidated[duplicate] = {original, duplicate}
    
    # Create a mapping of duplicates to originals
    mapping = {old: new for new, old_set in consolidated.items() for old in old_set}
    
    # Apply mapping to the column
    df[column_name] = df[column_name].replace(mapping)
    
    # Convert all values in the column back to strings (in case any values were converted to other types during the operation)
    df[column_name] = df[column_name].astype(str)
    
    # Iterate through each row and replace older cells with newer ones if there are differences
    for index, row in df.iterrows():
        duplicate_rows = df[df[column_name] == row[column_name]]
        if len(duplicate_rows) > 1:  # If there are duplicates
            newer_row = duplicate_rows.iloc[-1]  # Get the newer row
            for col in df.columns:  # Iterate through each column
                if row[col] != newer_row[col]:  # If the cell values are different
                    df.at[index, col] = newer_row[col]  # Replace with the newer value
    
    # Drop duplicate rows (keeping the last occurrence)
    df.drop_duplicates(subset=[column_name], keep='last', inplace=True)
    
    return df


In [ ]:
#to load the data into a new grouped table

#whichever new table should be on df2 and the old table should be df1
def main():
    df1 = load_data(' ')
    df2 = load_data(' ')
    combined_df = pd.concat([df1, df2], ignore_index=True)
    deduplicated_df = deduplicate_data(combined_df, 'Names')
    deduplicated_df.to_sql('groupcal6', engine, if_exists='replace', index=False)

if __name__ == '__main__':
    main()

In [ ]:
#use this to pull back the new grouped table and turn it into a csv file so it would go through the Consolidated Data Cleaner.ipynb


# filepath to where you want to extract the csv to so that you can use the next python script
filepath = ' '

# Extract directory path from the filepath
directory = os.path.dirname(filepath)

# Create the directory if it doesn't exist
os.makedirs(directory, exist_ok=True)

# Make a dataframe from the loaded grouped table data
df4=load_data('groupcal2')

# Save the grouped table into your desktop as a csv file that can be read for the python script
df4.to_csv(filepath, index=False)